In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("kernelsurvivor = 1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [23]:
from ipywidgets import Button, HBox, Layout, VBox,Label, HTML
import os
import time


download_bou = Button(
    description='Atrium',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Atrium',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

def saveToAtrium(b):
    Javascript("""jupyter.actions.call('jupyter-notebook:save-notebook');""")
    t = time.localtime()
    a = os.popen("""zip -r nsipremiere{:4d}{:02d}{:02d}{:02d}{:02d}{:02d}.zip /home/jovyan/Perso""".format(t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec))
    for ligne in a:
        print(ligne)
    j = open("/home/jovyan/.netrc","r")
    data = j.readlines()
    j.close()
    data = data[1]
    login = data.split(" ")[1].strip()
    
    j = open("/home/jovyan/datadav","w")
    j.write("cd {}\ncd document_library\nput /home/jovyan/nsipremiere{:4d}{:02d}{:02d}{:02d}{:02d}{:02d}.zip\nexit".format(login,t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec))
    j.close()
    a = os.popen("""cadaver -r datadav https://www.atrium-sud.fr/webdav""")
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    
download_bou.on_click(saveToAtrium)
display(HBox([download_bou]))

In [16]:
import time
t = time.localtime()
print(t)
print(t.tm_year,t.tm_mon,t.tm_mday)

time.struct_time(tm_year=2021, tm_mon=10, tm_mday=31, tm_hour=23, tm_min=28, tm_sec=4, tm_wday=6, tm_yday=304, tm_isdst=0)
2021 10 31
